🔹 DECISION ALGORITHM 3: Actor–Critic (Advanced Strategy Learning)

In [1]:
%pip install pennylane pennylane-lightning torch scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.


## 3A. Classical Actor–Critic

In [4]:
# TOPIC: Classical RL - Actor-Critic
import torch

actor = torch.nn.Sequential(
    torch.nn.Linear(4, 2),
    torch.nn.Softmax(dim=-1)
)

critic = torch.nn.Linear(4, 1)

opt = torch.optim.Adam(
    list(actor.parameters()) + list(critic.parameters()), lr=0.01
)

for episode in range(100):
    state = torch.rand(4)
    probs = actor(state)
    action = torch.multinomial(probs, 1)

    value = critic(state)
    reward = torch.randn(1)
    advantage = reward - value

    actor_loss = -torch.log(probs[action]) * advantage
    critic_loss = advantage.pow(2)

    loss = actor_loss + critic_loss
    opt.zero_grad()
    loss.backward()
    opt.step()


## 3B. Quantum Actor–Critic

Actor = quantum policy, Critic = quantum value

In [6]:
# TOPIC: Quantum RL - Quantum Actor-Critic
import pennylane as qml
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev, interface="torch")
def actor_circuit(state, weights):
    qml.AngleEmbedding(state, wires=[0,1])
    qml.BasicEntanglerLayers(weights, wires=[0,1])
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]

@qml.qnode(dev, interface="torch")
def critic_circuit(state, weights):
    qml.AngleEmbedding(state, wires=[0,1])
    qml.BasicEntanglerLayers(weights, wires=[0,1])
    return qml.expval(qml.PauliZ(0))

actor_w = torch.nn.Parameter(torch.randn(2, 2))
critic_w = torch.nn.Parameter(torch.randn(2, 2))

optimizer = torch.optim.Adam([actor_w, critic_w], lr=0.1)

for episode in range(100):
    state = torch.rand(2)
    out = actor_circuit(state, actor_w)
    probs = torch.softmax(torch.as_tensor(out), dim=0)
    action = torch.multinomial(probs, 1)

    value = critic_circuit(state, critic_w)
    reward = torch.randn(1)
    advantage = reward - value

    loss = -torch.log(probs[action]) * advantage + advantage.pow(2)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_21708\4011858914.py:25: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  probs = torch.softmax(torch.as_tensor(out), dim=0)


**Paper framing:**
“Quantum actor–critic architectures combine quantum policy learning with quantum value estimation.”